<a href="https://colab.research.google.com/github/Ut14/WebScrape-Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install langchain
# !pip install huggingface_hub
# !pip install sentence_transformers
# !pip install selenium beautifulsoup4
# !apt-get update
# !apt install chromium-chromedriver
# !apt-get install -y chromium-browser
# !pip install langchain-community langchain-core

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
!ls /usr/lib/chromium-browser/chromedriver

/usr/lib/chromium-browser/chromedriver


In [ ]:
options=webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

# Get HUGGINGFACEHUB_API_KEY

In [ ]:
import os
from getpass import getpass
HuggingFaceHub_API_KEY = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HuggingFaceHub_API_KEY

··········


In [ ]:
url = 'https://botpenguin.com'
driver.get(url)

# Get the page source
page_source = driver.page_source

In [ ]:
# Use BeautifulSoup to parse the page source
soup = BeautifulSoup(page_source, 'html.parser')

# Extract all text content
text_content = soup.get_text(separator='\n')

# Clean up the text content
cleaned_text = re.sub(r'\n\s*\n', '\n\n', text_content)  # Remove excessive new lines
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Replace multiple spaces with a single space

# Print the cleaned text content
print(cleaned_text)

 Free Chatbot maker | Chatbot for Website, WhatsApp | BotPenguin Why BotPenguin Product Solutions Pricing Partners Resources Login Get Started FREE Engage, Converse and Convert your visitors using AI Chatbot Agent Generate 10x more leads, solve up to 80% customer queries, engage 70% more visitors to earn 90% more revenue by automating business communication. Get Started FREE *FREE Forever plan. No Credit Card Required. BotPenguin is the best AI Chatbot maker platform. Create a Chatbot for WhatsApp, Website, Facebook Messenger, Telegram, WordPress & Shopify with BotPenguin - 100% FREE! Our chatbot creator helps with lead generation, appointment booking, customer support, marketing automation, WhatsApp & Facebook Automation for businesses. AI-powered No-Code chatbot maker with live chat plugin & ChatGPT integration. Mobile app Download iOS App Download Android App Mobile app Download iOS App Download Android App Platforms Microsoft Teams New Instagram New Whatsapp Telegram Facebook Websi

In [ ]:
driver.quit()

In [ ]:
with open('cleaned_text.txt', 'w') as f:
    f.write(cleaned_text)

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./cleaned_text.txt')
documents = loader.load()

In [ ]:
document_text = documents[0].page_content  # Access the text content
print(len(document_text))

2100


In [ ]:
#Text Splitter
# from langchain.text_splitter import CharacterTextSplitter
# text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)

In [ ]:
# print(f"Number of chunks: {len(texts)}")
# for i, text in enumerate(texts):
#     print(f"Chunk {i + 1} length: {len(text.page_content)}")

Number of chunks: 1
Chunk 1 length: 2098


In [ ]:
from langchain.docstore.document import Document

In [ ]:
# Function to split text into chunks
def split_text(text, chunk_size):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return [Document(page_content=chunk) for chunk in chunks] # Create Document objects for each chunk

# Split the text
texts= split_text(document_text, 1000)
len(texts)

3

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
# !pip install faiss-cpu

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

In [ ]:
query="Why is BotPenguin good?"
docs = db.similarity_search(query)

In [ ]:
print((str(docs[0].page_content)))

 Free Chatbot maker | Chatbot for Website, WhatsApp | BotPenguin Why BotPenguin Product Solutions Pricing Partners Resources Login Get Started FREE Engage, Converse and Convert your visitors using AI Chatbot Agent Generate 10x more leads, solve up to 80% customer queries, engage 70% more visitors to earn 90% more revenue by automating business communication. Get Started FREE *FREE Forever plan. No Credit Card Required. BotPenguin is the best AI Chatbot maker platform. Create a Chatbot for WhatsApp, Website, Facebook Messenger, Telegram, WordPress & Shopify with BotPenguin - 100% FREE! Our chatbot creator helps with lead generation, appointment booking, customer support, marketing automation, WhatsApp & Facebook Automation for businesses. AI-powered No-Code chatbot maker with live chat plugin & ChatGPT integration. Mobile app Download iOS App Download Android App Mobile app Download iOS App Download Android App Platforms Microsoft Teams New Instagram New Whatsapp Telegram Facebook Websi

# Create QnA Chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.1, "max_length":512})

In [ ]:
chain=load_qa_chain(llm, chain_type="stuff")

In [ ]:
print("Bot: Hello I am Bot. How can I help you?(type exit to quit)")
while True:
  query=input("User: ")
  if query.lower()=="exit":
    print("Bot: Bye Bye")
    break
  answer = chain.run(input_documents=docs, question=query)
  answer=re.sub(r'\n+', '\n', answer)
  print("Bot: ", answer)

Bot: Hello I am Bot. How can I help you?(type exit to quit)
User: exit
Bot: Bye Bye
